In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
import photutils as phot
c = SkyCoord('12 01 36.81 -65 08 49.22', unit=(u.hourangle, u.deg))

In [208]:
def gaussian_psf_extraction(image, pix2arcsec, beam_size, pos):
    import photutils as phot
    
    # Gaussian sigma in pixel unit
    psf = phot.psf.GaussianPSF(beam_size/pix2arcsec)
    flux_gaussian = phot.psf.psf_photometry(image, [pos], psf)
    
    return flux_gaussian

In [209]:
def Gaussian_extraction(position, radius, image, plate_scale, norm=True):
    # radius in arcsec
    # size is an array with the same size of the image array.  Or the image array will do the trick
    # plate scale is how many arcsec of a single pixel
    # position is the pixel coordinates where the 2D Gaussian should center on
    
    # Create a 2-D cartian grid for mapping 2-D Gaussian
    def gauss2d(x, y, sigmax, sigmay=None, norm=True):
        # sigmax needs to be in pixel coordinates
        if sigmay == None:
            sigmay = sigmax
        if norm:
            gauss = 1/(2*np.pi*sigmax*sigmay) * np.exp( -(x**2/2./sigmax**2 + y**2/2./sigmay**2) )
        else:
            gauss = np.exp( -(x**2/2./sigmax**2 + y**2/2./sigmay**2) )
        return gauss
    
    size_x, size_y = np.shape(image)
    grid_x, grid_y = np.meshgrid(np.linspace(0,size_x-1,size_x), np.linspace(0,size_y-1,size_y))
    grid_x_cen = grid_x - (size_x-1)/2.
    grid_y_cen = grid_y - (size_y-1)/2.

    grid_gauss2d = gauss2d(grid_x_cen, grid_y_cen, sigmax=radius/plate_scale, norm=norm)
    
    return grid_gauss2d

In [1]:
# path to files
pacs70 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342224922/level2/HPPPMAPB/hpacs1342224922_20hpppmapb_00_1431606963402.fits'
pacs100 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342224923/level2/HPPPMAPB/hpacs1342224923_20hpppmapb_00_1431608216039.fits'
pacs160 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342224923/level2/HPPPMAPR/hpacs1342224923_20hpppmapr_00_1431608216611.fits'
spire500 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342226633/level2/extdPLW/hspireplw1342226633_20pxmp_1431669349619.fits'
spire350 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342226633/level2/extdPMW/hspirepmw1342226633_20pxmp_1431669350070.fits'
spire250 = '/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342226633/level2/extdPSW/hspirepsw1342226633_20pxmp_1431669350548.fits'

In [27]:
data = fits.open(pacs160)

In [211]:
data = fits.open('/Users/yaolun/bhr71/data/photometry/herschel_photometry/1342224922/browseProduct/blue_JScanam_map/hpacs_25HPPJSMAPB_blue_1202_m6509_00_v1.0_1434550765178.fits')

## Get the pixel coordinates of the source from the physical coordinates

In [28]:
from astropy.coordinates import SkyCoord
c = SkyCoord('12 01 36.81 -65 08 49.22', unit=(u.hourangle, u.deg))
w = wcs.WCS(data[1].header)
data_pixcoord = w.wcs_world2pix(np.array([[c.ra.degree, c.dec.degree]]),1)[0]

In [213]:
import photutils as phot
psf = phot.psf.GaussianPSF(13.5/pix2arcsec)

## Get the pixel to physical conversion factor

In [29]:
cdelt1 = data[1].header['CDELT1']
cdelt2 = data[1].header['CDELT2']
if abs(cdelt1) != abs(cdelt2):
    print 'pixel scales are different in two axis. %f and %f' % (cdelt1, cdelt2)
pix2arcsec = abs(cdelt1)*3600

pixel scales are different in two axis. -0.000889 and 0.000889


In [7]:
im = data[1].data
print np.nansum(im)*3.6/4250
gauss2d_masked = Gaussian_extraction(data_pixcoord, 13.5, im, pix2arcsec, norm=False)
im_extracted = im * gauss2d_masked.T
print np.sum(im_extracted)

0.112582232335


NameError: name 'Gaussian_extraction' is not defined

In [220]:
def Gaussian_spire_extraction(filename, gauss_width, source_coord, spire=False):
    import numpy as np
    import matplotlib.pyplot as plt
    import astropy.constants as const
    from astropy.io import fits
    from astropy import wcs
    from astropy import units as u
    
    data = fits.open(filename)        
    w = wcs.WCS(data[1].header)
    data_pixcoord = w.wcs_world2pix(np.array([[source_coord.ra.degree, source_coord.dec.degree]]),1)[0]
    print data_pixcoord
    
    # Get the pixel to physical conversion factor
    cdelt1 = data[1].header['CDELT1']
    cdelt2 = data[1].header['CDELT2']
    if abs(cdelt1) != abs(cdelt2):
        print 'pixel scales are different in two axis. %f and %f' % (cdelt1, cdelt2)
    pix2arcsec = abs(cdelt1)*3600
    print pix2arcsec
    
    im = data[1].data
    
    if spire:
        # unit is MJy/sr. Convert it to Jy/pixel
        factor = (u.MJy / u.sr * (pix2arcsec * u.arcsec) ** 2 / u.pixel).to(u.Jy / u.pixel)
        im *= factor.value
    
    gauss2d_masked = Gaussian_extraction(data_pixcoord, gauss_width, im, pix2arcsec, norm=False)
    print gauss2d_masked.max()
    im_extracted = im * gauss2d_masked.T
    
    return im_extracted

In [123]:
from astropy.coordinates import SkyCoord
c = SkyCoord('12 01 36.81 -65 08 49.22', unit=(u.hourangle, u.deg))
print c.ra.degree, c.dec.degree
flux500 = Gaussian_spire_extraction(spire500, 40.5, c, spire=True)
print np.nansum(flux500)

180.403375 -65.1470055556
0.970568307629
47.3155192295


In [30]:
aperture = phot.CircularAperture((data_pixcoord[0], data_pixcoord[1]), r=31.8/pix2arcsec)
factor = (u.MJy / u.sr * (pix2arcsec * u.arcsec) ** 2 / u.pixel).to(u.Jy / u.pixel)
phot_table = phot.aperture_photometry(data[1].data, aperture)
print phot_table['aperture_sum']

 aperture_sum
             
-------------
296.708993089
